#Новичков Егор ЛР1

In [1]:
from pyspark import SparkContext, SparkConf
app_name = "Lab1"
conf = SparkConf().setAppName(app_name).setMaster('local[1]')
sc = SparkContext(conf=conf)

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


Импортим данные в РФС и проверяем

In [2]:
!hadoop fs -put /mnt/data /data


!hadoop fs -ls /data

Found 1 items
drwxr-xr-x   - root root          0 2022-12-05 17:02 /data/data


Проверим работоспособность

In [3]:
warandpeace = sc.textFile("warandsociety.txt")
print(warandpeace.count())

12851


Проверка на несуществующем файле

In [4]:
nilFile = sc.textFile("nil")

Выведем первые элементы

In [5]:
warandpeace.take(25) 

['Лев Николаевич Толстой',
 'Война и мир. Книга 1',
 '',
 'Война и мир – 1',
 '',
 ' ',
 ' http://www.lib.ru',
 '',
 'Аннотация ',
 '',
 'Роман Льва Толстого «Война и мир» лежит в основании величественного здания русской классической литературы. С непревзойденным мастерством Толстой воссоздал великую духом Россию – образы этой «книги на все времена» и сейчас пленяют свежестью чувств и щедростью души, искренностью страстей, силой и чистотой убеждений.',
 'В книгу вошли первый и второй тома романа.',
 '',
 'Лев Николаевич Толстой',
 'ВОЙНА И МИР',
 '',
 'Том 1',
 '',
 'ЧАСТЬ ПЕРВАЯ',
 '',
 '',
 'I',
 '',
 "– Еh bien, mon prince. Genes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous previens, que si vous ne me dites pas, que nous avons la guerre, si vous vous permettez encore de pallier toutes les infamies, toutes les atrocites de cet Antichrist (ma parole, j'y crois) – je ne vous connais plus, vous n'etes plus mon ami, vous n'etes plus мой в

Выведем первую строчку со словом "война"

In [6]:
linesWithWar = warandpeace.filter(lambda x: "война " in x) 
linesWithWar.first()

'– Нет, не был, но вот что мне пришло в голову, и я хотел вам сказать. Теперь война против Наполеона. Ежели б это была война за свободу, я бы понял, я бы первый поступил в военную службу; но помогать Англии и Австрии против величайшего человека в мире… это нехорошо…'

Замер времени

In [7]:
def time(f):  
    import time
    t = time.process_time()
    f()
    print(f"Elapsed time: {int((time.process_time() - t)*1e9)} ns")

In [9]:
linesWithWar.cache()
time(lambda: linesWithWar.count())
time(lambda: linesWithWar.count())

Elapsed time: 8948200 ns
Elapsed time: 9669000 ns


Найдем гистограмму слов

In [10]:
wordCounts = linesWithWar.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)) .reduceByKey(lambda a, b: a + b)

In [11]:
wordCounts.saveAsTextFile("warandpeace_histogram.txt")

## Подготовим методы для чтения данных (из вспомогательной методички)

In [12]:
from typing import NamedTuple
from datetime import datetime

def initStation(stations):
    class Station(NamedTuple):
        station_id: int
        name: str
        lat: float
        long: float
        dockcount: int
        landmark: str
        installation: str
    
    for station in stations:
        yield Station(
            station_id = int(station[0]),
            name = station[1],
            lat = float(station[2]),
            long = float(station[3]),
            dockcount = int(station[4]),
            landmark = station[5],
            installation = datetime.strptime(station[6], '%m/%d/%Y')
        )
        
def initTrip(trips):
    class Trip(NamedTuple):
        trip_id: int
        duration: int
        start_date: datetime
        start_station_name: str
        start_station_id: int
        end_date: datetime
        end_station_name: str
        end_station_id: int
        bike_id: int
        subscription_type: str
        zip_code: str
        
    for trip in trips:
        yield Trip(                             
            trip_id = int(trip[0]),
            duration = int(trip[1]) if trip[1] != '' else 0,
            start_date = datetime.strptime(trip[2], '%m/%d/%Y %H:%M') if trip[2] != '' else None,
            start_station_name = trip[3],
            start_station_id = int(trip[4]),
            end_date = datetime.strptime(trip[5], '%m/%d/%Y %H:%M') if trip[5] != '' else None,
            end_station_name = trip[6],
            end_station_id = int(trip[7]),
            bike_id = int(trip[8]),
            subscription_type = trip[9],
            zip_code = trip[10]
        )

In [13]:
def GetDataFromTable(data): # метод, вовзрающий только данные, без названия столбцов
    columns = data.first()
    table = data.filter(lambda row: row != columns)\
                .map(lambda row: row.split(","))
    return columns, table

Инициализация данных

In [16]:
tripData = sc.textFile("trips.csv")
stationData = sc.textFile("stations.csv")

In [17]:
tripColumns, trips = GetDataFromTable(tripData)
stationCoolumns, stations = GetDataFromTable(stationData)

In [18]:
tripsObjects = trips.mapPartitions(initTrip)
stationObjects = stations.mapPartitions(initStation)

# 1 Найти велосипед с максимальным временем пробега.

In [19]:
#                создаем пары:(id,пробег)               суммируем для каждого байка его пробеги   отсортируем по пробегу и выведем первый элемент
bike_with_top_duration = tripsObjects.map(lambda trip: (trip.bike_id, trip.duration)).reduceByKey(lambda a, b: a + b).top(1, key = lambda x: x[1])[0][0]
print(bike_with_top_duration)

535


# 2 Найти наибольшее геодезическое расстояние между станциями.

In [22]:
trips_stations = tripsObjects \
.filter(lambda trip: trip.start_station_id != trip.end_station_id) \
.keyBy(lambda trip: (trip.start_station_id, trip.end_station_id)) \
.mapValues(lambda trip: trip.duration)


query = trips_stations \
.aggregateByKey((0.0, 0.0), \
                lambda acc, value: (acc[0] + value, acc[1] + 1), \
                lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1]),) \
.mapValues(lambda values: values[0] / values[1])

query.map(lambda x: x[::-1]).top(1)

[(229914.0, (26, 16))]

# 3 Найти путь велосипеда с максимальным временем пробега через станции.

In [23]:
result = tripsObjects.filter(lambda x: x.bike_id == bike_with_top_duration).sortBy(lambda x: x.start_date).map(lambda x: (x.start_station_name, x.end_station_name)).first()
print(result)

('Post at Kearney', 'San Francisco Caltrain (Townsend at 4th)')


# 4 Найти количество велосипедов в системе.

In [24]:
# оставляем уникальные id
result = tripsObjects.map(lambda trip: trip.bike_id).distinct().count()
print(result)

700


# 5 Найти пользователей потративших на поездки более 3 часов.

In [25]:
users = tripsObjects.filter(lambda x: x.duration > (10800)).map(lambda x: x.zip_code).filter(lambda x: x != "").distinct()

users.take(20)

['58553',
 '94301',
 '94039',
 '94133',
 '93726',
 '94123',
 '4517',
 '29200',
 '45322',
 '94080',
 '92808',
 '5024',
 '89138',
 '11515',
 '28277',
 '34990',
 '94803',
 '92663',
 '94109',
 '91801']